In [1]:
# import packages

import gc
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format # disabling scientific notation in pandas
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lightgbm as lgb

gc.enable()

In [2]:
data = pd.read_csv("data/train_filtered.csv",dtype={'fullVisitorId':str}) # as I have no dedicated test data, this is all the data I have

In [ ]:
data['revenue_generated'] = (data['totals_transactionRevenue'] > 0).astype(int)
data['revenue_amount'] = data['totals_transactionRevenue']/(10**6)
data.drop(columns='totals_transactionRevenue', inplace=True)

In [ ]:
data.drop(columns='trafficSource_adwordsClickInfo.isVideoAd', inplace=True) ## ADD TO PREPROCESSING. Feature has a constant value and has thus no information

Add time features

In [ ]:
data.date = pd.to_datetime(data.date, format='%Y%m%d')
data['date_time'] = pd.to_datetime(data['visitStartTime'], unit='s')
data['date_dow'] = data['date_time'].dt.dayofweek
data['date_hours'] = data['date_time'].dt.hour
data['date_dom'] = data['date_time'].dt.day
data['month'] = data['date_time'].dt.month

In [ ]:
data.sort_values('date') # it is too little data to split it date-wise -- as then we would have only data for a month

In [ ]:
train, test = train_test_split(data, test_size=0.3)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
def calc_days_since_visit(row):
    if row['fullVisitorId'] == row['last_visit_id']:
        return (row['date'] - row['last_visit_date']).days
    else:
        return np.nan

In [ ]:
def get_delays(df):
    visit_dates = df[['fullVisitorId', 'date']]
    visit_dates = visit_dates.sort_values(['fullVisitorId', 'date'])
    visit_dates[['last_visit_id', 'last_visit_date']] = visit_dates.shift(+1)[['fullVisitorId', 'date']]
    visit_dates['days_since_visit'] = visit_dates.apply(calc_days_since_visit, axis=1)
    visit_dates.sort_index(inplace=True)
    
    return visit_dates['days_since_visit']

In [ ]:
train['days_since_visit'] = get_delays(train)
test['days_since_visit'] = get_delays(test)

In [ ]:
for df in [train, test]:
    df['days_since_visit'] = get_delays(df)

In [ ]:
# https://www.kaggle.com/prashantkikani/teach-lightgbm-to-sum-predictions-fe
def browser_mapping(x):
    browsers = ['chrome','safari','firefox','internet explorer','edge','opera','coc coc','maxthon','iron']
    if x in browsers:
        return x.lower()
    elif  ('android' in x) or ('samsung' in x) or ('mini' in x) or ('iphone' in x) or ('in-app' in x) or ('playstation' in x):
        return 'mobile browser'
    elif  ('mozilla' in x) or ('chrome' in x) or ('blackberry' in x) or ('nokia' in x) or ('browser' in x) or ('amazon' in x):
        return 'mobile browser'
    elif  ('lunascape' in x) or ('netscape' in x) or ('blackberry' in x) or ('konqueror' in x) or ('puffin' in x) or ('amazon' in x):
        return 'mobile browser'
    elif '(not set)' in x:
        return x
    else:
        return 'others'
    
    
def adcontents_mapping(x):
    if  ('google' in x):
        return 'google'
    elif  ('placement' in x) | ('placememnt' in x):
        return 'placement'
    elif '(not set)' in x or 'nan' in x:
        return x
    elif 'ad' in x:
        return 'ad'
    else:
        return 'others'
    
def source_mapping(x):
    if  ('google' in x):
        return 'google'
    elif  ('youtube' in x):
        return 'youtube'
    elif '(not set)' in x or 'nan' in x:
        return x
    elif 'yahoo' in x:
        return 'yahoo'
    elif 'facebook' in x:
        return 'facebook'
    elif 'reddit' in x:
        return 'reddit'
    elif 'bing' in x:
        return 'bing'
    elif 'quora' in x:
        return 'quora'
    elif 'outlook' in x:
        return 'outlook'
    elif 'linkedin' in x:
        return 'linkedin'
    elif 'pinterest' in x:
        return 'pinterest'
    elif 'ask' in x:
        return 'ask'
    elif 'siliconvalley' in x:
        return 'siliconvalley'
    elif 'lunametrics' in x:
        return 'lunametrics'
    elif 'amazon' in x:
        return 'amazon'
    elif 'mysearch' in x:
        return 'mysearch'
    elif 'qiita' in x:
        return 'qiita'
    elif 'messenger' in x:
        return 'messenger'
    elif 'twitter' in x:
        return 'twitter'
    elif 't.co' in x:
        return 't.co'
    elif 'vk.com' in x:
        return 'vk.com'
    elif 'search' in x:
        return 'search'
    elif 'edu' in x:
        return 'edu'
    elif 'mail' in x:
        return 'mail'
    elif 'ad' in x:
        return 'ad'
    elif 'golang' in x:
        return 'golang'
    elif 'direct' in x:
        return 'direct'
    elif 'dealspotr' in x:
        return 'dealspotr'
    elif 'sashihara' in x:
        return 'sashihara'
    elif 'phandroid' in x:
        return 'phandroid'
    elif 'baidu' in x:
        return 'baidu'
    elif 'mdn' in x:
        return 'mdn'
    elif 'duckduckgo' in x:
        return 'duckduckgo'
    elif 'seroundtable' in x:
        return 'seroundtable'
    elif 'metrics' in x:
        return 'metrics'
    elif 'sogou' in x:
        return 'sogou'
    elif 'businessinsider' in x:
        return 'businessinsider'
    elif 'github' in x:
        return 'github'
    elif 'gophergala' in x:
        return 'gophergala'
    elif 'yandex' in x:
        return 'yandex'
    elif 'msn' in x:
        return 'msn'
    elif 'dfa' in x:
        return 'dfa'
    elif '(not set)' in x:
        return '(not set)'
    elif 'feedly' in x:
        return 'feedly'
    elif 'arstechnica' in x:
        return 'arstechnica'
    elif 'squishable' in x:
        return 'squishable'
    elif 'flipboard' in x:
        return 'flipboard'
    elif 't-online.de' in x:
        return 't-online.de'
    elif 'sm.cn' in x:
        return 'sm.cn'
    elif 'wow' in x:
        return 'wow'
    elif 'baidu' in x:
        return 'baidu'
    elif 'partners' in x:
        return 'partners'
    else:
        return 'others'
    

In [ ]:
def process_device(df):
    print("process device ...")
    df['source_country'] = df['trafficSource_source'] + '_' + df['geoNetwork_country']
    df['campaign_medium'] = df['trafficSource_campaign'] + '_' + df['trafficSource_medium']
    df['browser_category'] = df['device_browser'] + '_' + df['device_deviceCategory']
    df['browser_os'] = df['device_browser'] + '_' + df['device_operatingSystem']
    return df


def custom(df):
    print('custom ...')
    df['device_deviceCategory_channelGrouping'] = df['device_deviceCategory'] + "_" + df['channelGrouping']
    df['channelGrouping_browser'] = df['device_browser'] + "_" + df['channelGrouping']
    df['channelGrouping_OS'] = df['device_operatingSystem'] + "_" + df['channelGrouping']
    
    for i in ['geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country','geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region','geoNetwork_subContinent']:
        for j in ['device_browser','device_deviceCategory', 'device_operatingSystem', 'trafficSource_source']:
            df[i + "_" + j] = df[i] + "_" + df[j]
    
    df['content_source'] = df['trafficSource_adContent'] + "_" + df['source_country']
    df['medium_source'] = df['trafficSource_medium'] + "_" + df['source_country']
    return df

In [ ]:
for df in [train, test]:

    df['device_browser'] = df['device_browser'].map(lambda x:browser_mapping(str(x).lower())).astype('str')
    df['trafficSource_adContent'] = df['trafficSource_adContent'].map(lambda x:adcontents_mapping(str(x).lower())).astype('str')
    df['trafficSource_source'] = df['trafficSource_source'].map(lambda x:source_mapping(str(x).lower())).astype('str')
    
    df = process_device(df)
    df = custom(df)

In [ ]:
excluded_features = [
    'date', 'fullVisitorId', 'visitStartTime'
]

categorical_features = [
    _f for _f in train.columns
    if (_f not in excluded_features) & (train[_f].dtype == 'object')
]

ToDo. `pd.factorize()`

In [ ]:
categorical_features

In [ ]:
for f in categorical_features:
    train[f], indexer = pd.factorize(train[f]) # https://lightgbm.readthedocs.io/en/latest/Advanced-Topics.html -- lgbm performs well with factorization instead of ohe
    test[f] = indexer.get_indexer(test[f])

In [ ]:
[c for c in X_train.columns if c not in categorical_features]

In [ ]:
excluded_features = ['revenue_generated', 'revenue_amount', 'fullVisitorId', 'visitStartTime', 'date_time', 'date']

In [ ]:
X_train = train.drop(columns=excluded_features)
y_train = train.revenue_generated

In [ ]:
X_test = test.drop(columns=excluded_features)
y_test = test.revenue_generated

In [ ]:
# X_train.to_csv('data/X_train.csv')
# y_train.to_csv('data/y_train.csv')
# X_test.to_csv('data/X_test.csv')
# y_test.to_csv('data/y_test.csv')